In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
import datetime
import tqdm
import importlib

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from databroker import catalog

In [ ]:
import bact_analysis
import bact_analysis.bba.calc
import bact_analysis.transverse.calc
import bact_analysis.transverse.distorted_orbit
import bact_analysis.transverse.process
import bact_analysis.utils.preprocess
import bact_analysis_bessyii.bba.preprocess_data

# BBA Analysis

This notebook should be the center for selecting the required data and presenting the results.
All computation or data rearangement shall be performed in libraries

In [ ]:
datetime.datetime.now().strftime('%Y %m %d %H:%M')

## Data selection, check and load

Define the catalog database. In case of questions have a look at https://blueskyproject.io/databroker/

In [ ]:
catalog_name = 'datascc_bba_test'

Standard large database for HZB

In [ ]:
catalog_name = 'heavy'

In [ ]:
db = catalog[catalog_name]

Define the universal id of the run

In [ ]:
uid = '6c3d31df-0714-4ea9-b7e0-64df3ff039c0'

Here an illustration how to search for measurements

* I first define the acceptable time range
* From this time range I use the one that have the nickname "bba" set

In [ ]:
from databroker.queries import TimeRange

In [ ]:
t_search =  db.search(TimeRange(since="2022-04-08", until="2022-04-12"))
possible_bba = t_search.search(dict(nickname="bba"))
for uid in possible_bba:
    run = db[uid]
    start = run.metadata["start"]    
    ts_start = datetime.datetime.fromtimestamp(int(start['time']))
    stop = run.metadata["stop"]
    if not stop:
        print(f'{uid} {ts_start} ----')
        continue
    ts_end = datetime.datetime.fromtimestamp(int(stop['time']))
    print(f'{uid} {ts_start} {ts_end}')

In [ ]:
uid = '20e55c8b-6804-4f5f-bf22-a5a6a8521e41'
# uid = '874631e7-fec9-4666-a112-8dfa2e42545c'

In [ ]:
# uid = 'e5c25993-feab-4820-b4c5-b6d27e97942a'

In [ ]:
 run = db[uid]

### loading data

In [ ]:
descriptor, = run.primary.metadata['descriptors']
descriptor;

In [ ]:
configuration = descriptor['configuration']
dt_configuration = configuration['dt']
list(configuration.keys()), list(dt_configuration.keys())

Loading dask seems to be the fastest approach

In [ ]:
start = datetime.datetime.now()
all_data_ = run.primary.to_dask()
end = datetime.datetime.now()

end - start

Now load all data

In [ ]:
for name, item in tqdm.tqdm(all_data_.items(), total=len(all_data_.variables)):
    item.load()

The bpm names should be stored as configuration values: this is not the case for old data

In [ ]:
bpm_names_in_config = False

if bpm_names_in_config:
    bpm_names = dt_configuration['data']['dt_bpm_waveform_names']
else:
    bpm_names = all_data_.dt_bpm_waveform_names.isel(time=0).values

In [ ]:
bpm_names_lc = [name.lower() for name in bpm_names]

In [ ]:
bpm_names_check = set(bpm_names_lc)

Check that all bpm's are in the lattice

In [ ]:
bpm_names_check.difference(beam_info.coords["pos"].values)

### Collapsing the number of different dimensions

Replace them with known lablled alternatives

In [ ]:
bpm_dims = bact_analysis_bessyii.bba.preprocess_data.replaceable_dims_bpm(
    all_data_, prefix="dt_", expected_length=len(bpm_names)
)

### Derive info on measurement

When were magnets strength was switched or magnet was reselected

In [ ]:
muxer_pc_current_change = bact_analysis.utils.preprocess.enumerate_changed_value(all_data_.dt_mux_power_converter_setpoint)
muxer_pc_current_change.name = "muxer_pc_current_change" 
muxer_or_pc_current_change = bact_analysis.utils.preprocess.enumerate_changed_value_pairs(all_data_.dt_mux_power_converter_setpoint, all_data_.dt_mux_selector_selected)
muxer_or_pc_current_change.name = "muxer_or_pc_current_change" 

### Combine all info to new xarray 

In [ ]:
replace_dims = {dim : 'bpm' for dim in bpm_dims}
# replace_dims.update({dim : 'pos' for dim in beam_dims})
all_data = all_data_.rename(replace_dims).assign_coords(bpm=list(bpm_names))
all_data;

In [ ]:
preprocessed = xr.merge([all_data, muxer_pc_current_change, muxer_or_pc_current_change])
preprocessed;

### Make data selectable per magnet

In [ ]:
importlib.reload(bact_analysis.utils.preprocess)
importlib.reload(bact_analysis.transverse.calc)
importlib.reload(bact_analysis_bessyii.bba.preprocess_data)

In [ ]:
type(preprocessed)

In [ ]:
rearranged = xr.concat(
    bact_analysis.utils.preprocess.reorder_by_groups(
        preprocessed,
        preprocessed.groupby(preprocessed.dt_mux_selector_selected),
        reordered_dim="name",
        dim_sel="time",
        new_indices_dim="step",
    ),
    dim="name",
)

In [ ]:
tune_data = xr.merge([rearranged.dt_tunes_hor_readback, rearranged.dt_tunes_vert_readback, rearranged.dt_mux_power_converter_setpoint, rearranged.dt_mux_power_converter_readback])

In [ ]:
tune_sel = tune_data.sel(name="Q2M1T1R")

In [ ]:
from scipy.optimize import lsq_linear

In [ ]:
def fit_tune_shift(dI, tune):
    X = np.ones((2,) + dI.shape, dtype=np.float_)
    X[1, :] = dI
    res = lsq_linear(X.T, tune)
    return res

In [ ]:
res = fit_tune_shift(tune_sel.dt_mux_power_converter_setpoint, tune_sel.dt_tunes_hor_readback)

In [ ]:
res

In [ ]:
x = np.linspace(-2.2, 2.2)
t_f = res.x[1] * x
y = res.x[0] + t_f
t_f_m = tune_sel.dt_tunes_hor_readback  - res.x[0]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=[12, 6])
ax1, ax2= axes
line, = ax1.plot(tune_sel.dt_mux_power_converter_setpoint, tune_sel.dt_tunes_hor_readback, 'x')
ax1.plot(x, y, '-', color=line.get_color())
ax1.set_xlabel(r"\Delta I [A]")
ax1.set_ylabel(r"f [kHz]")
ax2.plot(tune_sel.dt_mux_power_converter_setpoint, t_f_m, 'x', color=line.get_color())
ax2.plot(x, t_f, '-', color=line.get_color())
ax2.set_xlabel(r"\Delta I [A]")
ax2.set_ylabel(r"\Delta t [kHz]")


In [ ]:
measurement_vars = dict(dt_beam_orbit_x='x_pos', dt_beam_orbit_y='y_pos', dt_mux_power_converter_setpoint='excitation')
redm4proc = rearranged[list(measurement_vars.keys())].rename_vars(**measurement_vars).sel(pos=bpm_names_lc)
redm4proc;

## Processing data

### Using model information

This model information is obtained here from the model produced by the digital shadow. 

Should be replaced by a standard lattice from database

In [ ]:
orb_dist_vars = dict(dt_beam_twiss_beta_x='beta_x', dt_beam_twiss_beta_y='beta_y', dt_beam_twiss_nu_x='nu_x', dt_beam_twiss_nu_y='nu_y')

In [ ]:
magnet_name = "Q1M1D1R"

selected_magnet_all_pos = dict(name=magnet_name, step=0)
selected_magnet_position = dict(pos=magnet_name.lower())

In [ ]:
selected_model_ = rearranged.sel(selected_magnet_all_pos)[list(orb_dist_vars.keys())]#.rename_vars(**orb_dist_vars)
selected_model_;

In [ ]:
beam_info

In [ ]:
(dim,) = selected_model_.dt_beam_twiss_beta_x.dims
selected_model = xr.Dataset(
    dict(
        beta=xr.DataArray(
            data=[
                selected_model_.dt_beam_twiss_beta_x,
                selected_model_.dt_beam_twiss_beta_y,
            ],
            dims=["plane", dim],
            coords=[["x", "y"], selected_model_.dt_beam_twiss_beta_x.coords[dim]],
        ),
        mu=xr.DataArray(
            data=[
                selected_model_.dt_beam_twiss_nu_x,
                selected_model_.dt_beam_twiss_nu_y,
            ],
            dims=["plane", dim],
            coords=[["x", "y"], selected_model_.dt_beam_twiss_nu_x.coords[dim]],
        ),
    )
).merge(dict(ds=beam_info.ds))
selected_model

In [ ]:
selected_model.to_netcdf('bessii_twiss_tracy.nc')

In [ ]:
selected_model_for_magnet = selected_model.sel(selected_magnet_position)
selected_model_for_magnet

In [ ]:
if False:
    measurement_vars = dict(dt_bpm_waveform_x_pos='x_pos', dt_bpm_waveform_y_pos='y_pos', dt_mux_power_converter_setpoint='excitation')
    selected_measurement = rearranged.sel(name=magnet_name)[list(measurement_vars.keys())].rename_vars(**measurement_vars)#.rename(bpm='pos')

In [ ]:
if True:
    measurement_vars = dict(dt_beam_orbit_x='x_pos', dt_beam_orbit_y='y_pos', dt_mux_power_converter_setpoint='excitation')
    selected_measurement = rearranged.sel(name=magnet_name)[list(measurement_vars.keys())].rename_vars(**measurement_vars).sel(pos=bpm_names_lc)

In [ ]:
selected_measurement

In [ ]:
theta = 1e-5

### rearranged

In [ ]:
selected_model

In [ ]:
importlib.reload(bact_analysis.transverse.distorted_orbit)

In [ ]:
orbit = bact_analysis.transverse.distorted_orbit.closed_orbit_distortion(
    selected_model.sel(plane="x"),
    selected_model_for_magnet.sel(plane="x"),
    theta,
    scale_tune=1,
    scale_phase_advance=2 * np.pi,
)
orbit;

In [ ]:
orbit

In [ ]:
orbit_at_bpm = orbit.sel(pos=bpm_names_lc)

In [ ]:
plt.plot(
    beam_info.ds, orbit, '-',
    beam_info.ds.sel(dict(pos=bpm_names_lc)), orbit_at_bpm, '.'
)

In [ ]:
res = bact_analysis.transverse.calc.derive_angle(orbit=orbit_at_bpm, excitation=selected_measurement.excitation, measurement=selected_measurement.x_pos)
res.sel(dict(parameter='scaled_angle'))

In [ ]:
ds = beam_info.ds.sel(pos=bpm_names_lc)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[8, 6])
ax.plot(ds, res.sel(dict(result='value',parameter=bpm_names_lc))*1e6 )

In [ ]:
ds = all_data.dt_bpm_waveform_ds.isel(time=0)
mean_orbit = res.sel(result='value',parameter=bpm_names_lc).rename(parameter='pos')
mean_orbit.coords

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[16, 8], sharex=True)
ax, ax_diff = axes
(dim,) = selected_measurement.excitation.dims

pscale = 1000

ax.plot(ds, mean_orbit * pscale, "k-", linewidth=2)

scaled_angle = res.sel(dict(parameter="scaled_angle"))

for step in selected_measurement.coords[dim]:
    if step == 6:
        #break
        pass
    model_scale = scaled_angle * selected_measurement.excitation[step]
    sel = selected_measurement.x_pos.sel(dict(step=step))
    (line,) = ax.plot(ds, sel.values * pscale)
    ax_diff.plot(
        ds, (sel - mean_orbit) * pscale, ".-", color=line.get_color(), linewidth=0.1
    )
    ax_diff.plot(
        ds, orbit_at_bpm * pscale * model_scale,"+",
        beam_info.ds, orbit * pscale * model_scale,"-",
        color=line.get_color(),linewidth=0.1
    )
    
ax.set_xlabel('ds [m]')
ax.set_ylabel('x, y [mm]')
ax_diff.set_ylabel('dx, dy [mm]');


## Test processing capabilities

In [ ]:
# d = {name: item for name, item in red4proc.coords.items()}

In [ ]:
#red4proc.x_pos.expand_dims?

In [ ]:
importlib.reload( bact_analysis.transverse.distorted_orbit)
importlib.reload( bact_analysis.transverse.calc)
importlib.reload( bact_analysis.transverse.process)

In [ ]:
start = datetime.datetime.now()
result = {
    name: item for name, item in 
    tqdm.tqdm(bact_analysis.transverse.process.process_all_gen(selected_model, redm4proc, redm4proc.coords['name'].values, bpm_names=bpm_names_lc, theta=1e-5), 
              total=len(redm4proc.coords['name']))
}
end  = datetime.datetime.now()
end - start

In [ ]:
rds = bact_analysis.transverse.process.combine_all(result)

In [ ]:
rds.orbit.attrs

In [ ]:
quad_names = [str(name.values) for name in rds.coords["name"]]
quad_names_lc = [name.lower() for name in quad_names]

In [ ]:
beam_info.ds.sel(pos=quad_names_lc)
rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value');

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[20, 16], sharex=True)
ax_x, ax_y = axes
a_scale = rds.orbit.attrs['theta'] * 1e6
ax_x.errorbar(
    beam_info.ds.sel(pos=quad_names_lc), rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value', plane='x') * a_scale, 
    yerr=rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='error', plane='x')*a_scale, 
    fmt= '.'
)
ax_y.errorbar(
    beam_info.ds.sel(pos=quad_names_lc), rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value', plane='y') * a_scale, 
    yerr=rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='error', plane='y')*a_scale, 
    fmt= '.'
)
ax_y.set_xlabel('s [m]')
ax_x.set_ylabel('x [$\mu rad$]')
ax_y.set_ylabel('y [$\mu$ rad]')


In [ ]:
import bact2_bessyii.magnets
importlib.reload(bact2_bessyii.magnets)

In [ ]:
import bact_analysis_bessyii.bba.calc
importlib.reload(bact_analysis_bessyii.bba.calc)

In [ ]:
offsets = bact_analysis_bessyii.bba.calc.angles_to_offset_all(rds, names=quad_names)
offsets;

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[20, 16], sharex=True)
ax_x, ax_y = axes
a_scale = 1e3
ax_x.errorbar(
    beam_info.ds.sel(pos=quad_names_lc),
    offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
    fmt=".",
)
ax_y.errorbar(
    beam_info.ds.sel(pos=quad_names_lc),
    offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".",
)
ax_y.set_xlabel("s [m]")
ax_x.set_ylabel("x [mm]")
ax_y.set_ylabel("y [mm]")